# Laserchicken tutorial
This document shows how Laserchicken can be used to calculate features that describe a point cloud.

## Load a las file

In [1]:
from laserchicken.read_las import read
point_cloud = read('testdata/AHN3.las')

In [2]:
point_cloud

## Normalize data
Adds the attribute 'normaiized_height' to the point cloud. Note that data is added to the given point cloud  in place 

In [3]:
from laserchicken.normalization import normalize
normalize(point_cloud)

{'vertex': {'x': {'type': 'float64',
   'data': array([131999.984125, 131997.625125, 131998.547125, ..., 132501.531125,
          132501.391125, 132500.641125])},
  'raw_classification': {'type': 'uint8',
   'data': array([9, 9, 9, ..., 9, 9, 9], dtype=uint8)},
  'gps_time': {'type': 'float64',
   'data': array([78563787.97322202, 78563787.93570042, 78563787.93571067, ...,
          78563778.28828931, 78563778.3107884 , 78563778.32578015])},
  'z': {'type': 'float64',
   'data': array([-0.34100002, -0.26600002, -0.26000002, ..., -0.24100002,
          -0.24100002, -0.24000002])},
  'intensity': {'type': 'uint16',
   'data': array([ 41, 152,  12, ...,  10,  15,  10], dtype=uint16)},
  'y': {'type': 'float64',
   'data': array([549718.375, 549723.875, 549722.875, ..., 549966.313, 549964.875,
          549964.563])},
  'normalized_height': {'type': 'float64',
   'data': array([1.236, 1.311, 1.317, ..., 1.336, 1.336, 1.337])}},
 'log': [{'time': datetime.datetime(2020, 1, 28, 10, 47, 55, 2

In [4]:
point_cloud

{'vertex': {'x': {'type': 'float64',
   'data': array([131999.984125, 131997.625125, 131998.547125, ..., 132501.531125,
          132501.391125, 132500.641125])},
  'raw_classification': {'type': 'uint8',
   'data': array([9, 9, 9, ..., 9, 9, 9], dtype=uint8)},
  'gps_time': {'type': 'float64',
   'data': array([78563787.97322202, 78563787.93570042, 78563787.93571067, ...,
          78563778.28828931, 78563778.3107884 , 78563778.32578015])},
  'z': {'type': 'float64',
   'data': array([-0.34100002, -0.26600002, -0.26000002, ..., -0.24100002,
          -0.24100002, -0.24000002])},
  'intensity': {'type': 'uint16',
   'data': array([ 41, 152,  12, ...,  10,  15,  10], dtype=uint16)},
  'y': {'type': 'float64',
   'data': array([549718.375, 549723.875, 549722.875, ..., 549966.313, 549964.875,
          549964.563])},
  'normalized_height': {'type': 'float64',
   'data': array([1.236, 1.311, 1.317, ..., 1.336, 1.336, 1.337])}},
 'log': [{'time': datetime.datetime(2020, 1, 28, 10, 47, 55, 2

## Filter points inside a polygon
We filter points within a polygon

In [5]:
from laserchicken.spatial_selections import points_in_polygon_wkt
polygon = "POLYGON(( 131963.984125 549718.375000," + \
                   " 132000.000125 549718.375000," + \
                   " 132000.000125 549797.063000," + \
                   " 131963.984125 549797.063000," + \
                   " 131963.984125 549718.375000))"
points_in_area = points_in_polygon_wkt(point_cloud, polygon)
point_cloud = points_in_area

## Filter points by attribute threshold

In [6]:
from laserchicken.select import select_above, select_below
points_below_1_meter = select_below(point_cloud, 'normalized_height', 1)
points_above_1_meter = select_above(point_cloud, 'normalized_height', 1)

## Compute neighbors
Computer neighbors for every point in the point cloud. This means that the target point cloud should contain the same points as the environment point cloud. The target point cloud contains the point at which the neighbors should be calculated.

The code below finds for every target point, the neighbors within a Spherical neighborhood with radius of 5 meters.

In [7]:
from laserchicken.compute_neighbors import compute_neighborhoods
from laserchicken.volume_specification import Sphere
targets = point_cloud
volume = Sphere(5)
neighbors = compute_neighborhoods(point_cloud, targets, volume)

## Calculate features

In [8]:
from laserchicken.feature_extractor import compute_features
compute_features(point_cloud, neighbors, targets, ['std_z','mean_z','slope'], volume)

Cylinder size in Bytes: 1214225560.6124551
Memory size in Bytes: 16982044672
Start tree creation
Done with env tree creation
Done with target tree creation
Extracting feature(s) "['eigenv_1', 'eigenv_2', 'eigenv_3', 'normal_vector_1', 'normal_vector_2', 'normal_vector_3', 'slope']"Extracting feature(s) "['eigenv_1', 'eigenv_2', 'eigenv_3', 'normal_vector_1', 'normal_vector_2', 'normal_vector_3', 'slope']" took 0.22 seconds
Extracting feature(s) "['mean_z', 'std_z', 'coeff_var_z']"Extracting feature(s) "['mean_z', 'std_z', 'coeff_var_z']" took 0.19 seconds
The following unrequested features were calculated as a side effect, but will not be returned: ['normal_vector_3', 'normal_vector_2', 'normal_vector_1', 'eigenv_3', 'eigenv_2', 'eigenv_1', 'coeff_var_z']


## Calculate parameterized features
We add 4 parameterized features.

In [9]:
from laserchicken.feature_extractor import register_new_feature_extractor
from laserchicken.feature_extractor.band_ratio_feature_extractor import BandRatioFeatureExtractor
register_new_feature_extractor(BandRatioFeatureExtractor(None,1,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(1,2,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(2,None,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(None,0,data_key='z'))

Print a list of all available feature names and notice the parameterized features we just registered, 'band_ratio_1<z<2'.

In [10]:
from laserchicken.feature_extractor import list_feature_names
sorted(list_feature_names())

['band_ratio_1<normalized_height<2',
 'band_ratio_2<normalized_height',
 'band_ratio_2<normalized_height<3',
 'band_ratio_3<normalized_height',
 'band_ratio_normalized_height<1',
 'band_ratio_z<0',
 'coeff_var_intensity',
 'coeff_var_normalized_height',
 'coeff_var_z',
 'density_absolute_mean_normalized_height',
 'density_absolute_mean_z',
 'echo_ratio',
 'eigenv_1',
 'eigenv_2',
 'eigenv_3',
 'entropy_normalized_height',
 'entropy_z',
 'kurto_normalized_height',
 'kurto_z',
 'max_intensity',
 'max_normalized_height',
 'max_z',
 'mean_intensity',
 'mean_normalized_height',
 'mean_z',
 'median_normalized_height',
 'median_z',
 'min_intensity',
 'min_normalized_height',
 'min_z',
 'normal_vector_1',
 'normal_vector_2',
 'normal_vector_3',
 'perc_100_normalized_height',
 'perc_100_z',
 'perc_10_normalized_height',
 'perc_10_z',
 'perc_11_normalized_height',
 'perc_11_z',
 'perc_12_normalized_height',
 'perc_12_z',
 'perc_13_normalized_height',
 'perc_13_z',
 'perc_14_normalized_height',
 

## Recalculate neighbors and calculate the parameterized feature
Neighbors is a generator and can only be iterated once. We therefore recalculate them here. If we would want to do multiple calculations without recalculating the neighbors, we could copy the neighbors to a list. This is not done by default because neighbors can quickly grow quite large so that available RAM unnecessarily becomes the bottle neck.

In [11]:
from laserchicken.volume_specification import InfiniteCylinder
cylinder = InfiniteCylinder(5)
neighborhoods = compute_neighborhoods(point_cloud, targets, cylinder)
compute_features(point_cloud, neighborhoods, targets, ['band_ratio_1<normalized_height<2'], cylinder)

Cylinder size in Bytes: 1214225560.6124551
Memory size in Bytes: 16982044672
Start tree creation
Done with env tree creation
Done with target tree creation
Extracting feature(s) "['band_ratio_1<normalized_height<2']"Extracting feature(s) "['band_ratio_1<normalized_height<2']" took 0.08 seconds


## Write result to ply file

In [12]:
from laserchicken.write_ply import write
write(point_cloud, 'my_output.ply')